<a href="https://colab.research.google.com/github/Mrcl3/O6EQTwpkNwvulbOa/blob/master/talent_search_with_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [197]:
!pip install unidecode
!pip install gensim
!pip install transformers
!pip install sentence-transformers



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [198]:
import numpy as np
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from unidecode import unidecode
import string
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Read the data

In [199]:
df = pd.read_csv('/content/drive/MyDrive/Aprziva/O6EQTwpkNwvulbOa/potential-talents - Aspiring human resources - seeking human resources.csv')

## Define the sentence to be checked

# Preprocessing

In [200]:
df.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [201]:
df = df.drop_duplicates(subset=['job_title', 'location', 'connection']).reset_index(drop=True)
df.tail(55)

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN
6,7,Student at Humber College and Aspiring Human R...,Kanada,61,NaN
7,8,HR Senior Specialist,San Francisco Bay Area,500+,NaN
8,10,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,NaN
9,11,Student at Chapman University,"Lake Forest, California",2,NaN


In [202]:
df.shape

(53, 5)

In [203]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from unidecode import unidecode

def preprocess(sentence):
    # Converting the words to lowercase
    sentence = sentence.lower()
    
    # Collecting a list of stop words from nltk and punctuation from the string class
    stopset = stopwords.words('english') + list(string.punctuation)
    
    # Remove stop words and punctuations from the string
    words = word_tokenize(sentence)
    words = [i for i in words if i.lower() not in stopset and not any(char in string.punctuation for char in i)]
    new_sentence = " ".join(words)
    
    # Remove non-ASCII characters
    new_sentence = unidecode(new_sentence)
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(new_sentence)
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return " ".join(words)



In [204]:
df['job_title'] = df['job_title'].apply(preprocess)


In [205]:
df['job_title'] 

0     2019 bauer college business graduate magna cum...
1     native english teacher epik english program korea
2                  aspiring human resource professional
3                   people development coordinator ryan
4          advisory board member celal bayar university
5                    aspiring human resource specialist
6     student humber college aspiring human resource...
7                                  hr senior specialist
8       seeking human resource hris generalist position
9                            student chapman university
10    svp chro marketing communication csr officer e...
11    human resource coordinator intercontinental bu...
12    aspiring human resource management student see...
13                   seeking human resource opportunity
14    experienced retail manager aspiring human reso...
15      human resource staffing recruiting professional
16                  human resource specialist luxottica
17    director human resource north america grou

# Word2Vec model

In [206]:
sentences = []
for item in df['job_title'].values:
  # Tokenize the sentence into a list of words
  words = nltk.word_tokenize(item)
    
  # Add the list of words to the sentences list
  sentences.append(words)
  model= Word2Vec(sentences,min_count=1,vector_size=300,workers=4)



In [207]:
print(model)

Word2Vec<vocab=172, vector_size=300, alpha=0.025>


In [208]:
model.save("word2vec.model")
model.save("model.bin")

In [209]:
model = Word2Vec.load('model.bin')

# Using the model to calculate similarities

In [210]:
similarity_array = []
# Define two example sentences
sentence2 = "aspiring human resources"
for item in df['job_title'].values:
  # Tokenize the sentences into lists of words
  tokens1 = nltk.word_tokenize(item)
  tokens2 = nltk.word_tokenize(sentence2)

  # Calculate the similarity between the two sentences
  similarity = model.wv.n_similarity(tokens1, tokens2)
  similarity_array.append(similarity)
df['similarities'] = similarity_array

In [211]:
df.tail(5)

,id,job_title,location,connection,fit,similarities
48,100,aspiring human resource manager graduating may...,"Cape Girardeau, Missouri",103,NaN,0.564992
49,101,human resource generalist loparex,"Raleigh-Durham, North Carolina Area",500+,NaN,0.395508
50,102,business intelligence analytics traveler,Greater New York City Area,49,NaN,0.009410
51,103,always set success,Greater Los Angeles Area,500+,NaN,-0.001944
52,104,director administration excellence logging,"Katy, Texas",500+,NaN,0.092985


# Basic fit ranking (similarities + connection scaled to (0,1)

In [212]:
from sklearn.preprocessing import MinMaxScaler

#Select the input features for ranking
features = ['similarities', 'connection']

# Replace '500+' with 500 in the 'connections' column
df['connection'] = df['connection'].replace('500+ ', '500')

df['connection'] = pd.to_numeric(df['connection'])
print(df['connection'])
# Scale the 'connections' column to a range of 0 to 1
scaler = MinMaxScaler()
df['connection'] = scaler.fit_transform(df['connection'].values.reshape(-1, 1))

# Normalize the remaining features to a range of 0 to 1
df[features] = scaler.fit_transform(df[features])

# Calculate the ranking score
df['ranking'] = 0.8*df['similarities'] * 0.2*df['connection']

# Normalize the ranking score to a range of 0 to 1
df['ranking'] = scaler.fit_transform(df['ranking'].values.reshape(-1, 1))

# Print the ranked candidates
print(df[['job_title', 'ranking']])


0      85
1     500
2      44
3     500
4     500
5       1
6      61
7     500
8     500
9       2
10    500
11    500
12    500
13    390
14     57
15    500
16    500
17    500
18     82
19    500
20      5
21      7
22     16
23    500
24    212
25    409
26    500
27      7
28     52
29    455
30    174
31    268
32     50
33    500
34      4
35     40
36     18
37    349
38    155
39     39
40     64
41      9
42    415
43     57
44     19
45     71
46      4
47     48
48    103
49    500
50     49
51    500
52    500
Name: connection, dtype: int64
                                            job_title   ranking
0   2019 bauer college business graduate magna cum...  0.098715
1   native english teacher epik english program korea  0.012769
2                aspiring human resource professional  0.102696
3                 people development coordinator ryan  0.130708
4        advisory board member celal bayar university  0.194983
5                  aspiring human resource specialist  

In [213]:
df.head()

,id,job_title,location,connection,fit,similarities,ranking
0,1,2019 bauer college business graduate magna cum...,"Houston, Texas",0.168337,NaN,0.460152,0.098715
1,2,native english teacher epik english program korea,Kanada,1.000000,NaN,0.010020,0.012769
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",0.086172,NaN,0.935151,0.102696
3,4,people development coordinator ryan,"Denton, Texas",1.000000,NaN,0.102564,0.130708
4,5,advisory board member celal bayar university,"İzmir, Türkiye",1.000000,NaN,0.153000,0.194983


# Job titles ranking

In [214]:
# Sort the DataFrame in ascending order based on a column
df_sorted = df.sort_values('ranking', ascending=False)

# Print the sorted DataFrame
print(df_sorted[['job_title', 'ranking', 'similarities', 'connection']])

                                            job_title   ranking  similarities  \
12  aspiring human resource management student see...  1.000000      0.784684   
16                human resource specialist luxottica  0.716094      0.561908   
49                  human resource generalist loparex  0.703923      0.552358   
8     seeking human resource hris generalist position  0.658423      0.516655   
13                 seeking human resource opportunity  0.580764      0.584583   
25                 human conflict policy compensation  0.571468      0.548438   
26                   human resource generalist schwan  0.567157      0.445040   
17  director human resource north america groupe b...  0.561285      0.440431   
15    human resource staffing recruiting professional  0.541803      0.425144   
11  human resource coordinator intercontinental bu...  0.535450      0.420159   
23  nortia staffing seeking human resource payroll...  0.509338      0.399670   
42  seeking human resource o

# Try with BERT

In [215]:
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel, BertTokenizer

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Selected words for similarity comparison
selected_words = ['aspiring', 'human', 'resource']

# Tokenize and encode all sentences and selected words together
tokenized_texts = tokenizer(df['job_title'].tolist(), padding=True, truncation=True, return_tensors='pt')
tokenized_selected_words = tokenizer(selected_words, padding=True, truncation=True, return_tensors='pt')

# Obtain BERT embeddings for all tokens
with torch.no_grad():
    outputs = model(**tokenized_texts)
    sentence_embeddings = outputs.last_hidden_state[:, 0, :].numpy()  # Sentence embeddings

    outputs_selected_words = model(**tokenized_selected_words)
    word_embeddings = outputs_selected_words.last_hidden_state[:, 0, :].numpy()  # Word embeddings

# Calculate cosine similarity between sentence embedding and word embeddings
similarity_scores = []
for i in range(len(df)):
    sentence_embedding = sentence_embeddings[i]
    scores = {}
    for j, word in enumerate(selected_words):
        word_embedding = word_embeddings[j]
        score = cosine_similarity(sentence_embedding.reshape(1, -1), word_embedding.reshape(1, -1)).item()
        scores[word] = score
    similarity_scores.append(scores)

# Reshape similarity scores and convert to DataFrame
similarity_scores = pd.DataFrame(similarity_scores, index=range(len(df)))
similarity_scores.columns = selected_words

# Merge similarity scores with the original DataFrame
df = pd.concat([df, similarity_scores], axis=1)

print(df)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


     id                                          job_title  \
0     1  2019 bauer college business graduate magna cum...   
1     2  native english teacher epik english program korea   
2     3               aspiring human resource professional   
3     4                people development coordinator ryan   
4     5       advisory board member celal bayar university   
5     6                 aspiring human resource specialist   
6     7  student humber college aspiring human resource...   
7     8                               hr senior specialist   
8    10    seeking human resource hris generalist position   
9    11                         student chapman university   
10   12  svp chro marketing communication csr officer e...   
11   13  human resource coordinator intercontinental bu...   
12   27  aspiring human resource management student see...   
13   28                 seeking human resource opportunity   
14   66  experienced retail manager aspiring human reso...   
15   67 

In [216]:
df.head()

,id,job_title,location,connection,fit,similarities,ranking,aspiring,human,resource
0,1,2019 bauer college business graduate magna cum...,"Houston, Texas",0.168337,NaN,0.460152,0.098715,0.802823,0.849149,0.811189
1,2,native english teacher epik english program korea,Kanada,1.000000,NaN,0.010020,0.012769,0.852012,0.874882,0.874288
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",0.086172,NaN,0.935151,0.102696,0.760295,0.807430,0.757689
3,4,people development coordinator ryan,"Denton, Texas",1.000000,NaN,0.102564,0.130708,0.853755,0.899812,0.868571
4,5,advisory board member celal bayar university,"İzmir, Türkiye",1.000000,NaN,0.153000,0.194983,0.794921,0.843250,0.816467


In [217]:
df['avarage'] = (df['aspiring'] + df['human'] + df['resource'])/3

In [218]:
df.head()

,id,job_title,location,connection,fit,similarities,ranking,aspiring,human,resource,avarage
0,1,2019 bauer college business graduate magna cum...,"Houston, Texas",0.168337,NaN,0.460152,0.098715,0.802823,0.849149,0.811189,0.821054
1,2,native english teacher epik english program korea,Kanada,1.000000,NaN,0.010020,0.012769,0.852012,0.874882,0.874288,0.867061
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",0.086172,NaN,0.935151,0.102696,0.760295,0.807430,0.757689,0.775138
3,4,people development coordinator ryan,"Denton, Texas",1.000000,NaN,0.102564,0.130708,0.853755,0.899812,0.868571,0.874046
4,5,advisory board member celal bayar university,"İzmir, Türkiye",1.000000,NaN,0.153000,0.194983,0.794921,0.843250,0.816467,0.818213


In [219]:
from sklearn.preprocessing import MinMaxScaler

#Select the input features for ranking
features = ['avarage', 'connection']

# Replace '500+' with 500 in the 'connections' column
df['connection'] = df['connection'].replace('500+ ', '500')

df['connection'] = pd.to_numeric(df['connection'])

# Scale the 'connections' column to a range of 0 to 1
scaler = MinMaxScaler()
df['connection'] = scaler.fit_transform(df['connection'].values.reshape(-1, 1))

# Normalize the remaining features to a range of 0 to 1
df[features] = scaler.fit_transform(df[features])

# Calculate the ranking score
df['ranking_BERT'] = 0.8*df['avarage'] * 0.2*df['connection']

# Normalize the ranking score to a range of 0 to 1
df['ranking_BERT'] = scaler.fit_transform(df['ranking_BERT'].values.reshape(-1, 1))

# Print the ranked candidates
print(df[['job_title', 'ranking','ranking_BERT']])


                                            job_title   ranking  ranking_BERT
0   2019 bauer college business graduate magna cum...  0.098715      0.094319
1   native english teacher epik english program korea  0.012769      0.745856
2                aspiring human resource professional  0.102696      0.032324
3                 people development coordinator ryan  0.130708      0.774029
4        advisory board member celal bayar university  0.194983      0.548840
5                  aspiring human resource specialist  0.000000      0.000000
6   student humber college aspiring human resource...  0.097855      0.078014
7                                hr senior specialist  0.086114      0.691845
8     seeking human resource hris generalist position  0.658423      0.681701
9                          student chapman university  0.000181      0.001476
10  svp chro marketing communication csr officer e...  0.148835      0.597022
11  human resource coordinator intercontinental bu...  0.535450 

In [220]:
# Sort the DataFrame in ascending order based on a column
df_sorted = df.sort_values('ranking', ascending=False)

# Print the sorted DataFrame
print(df_sorted[['job_title', 'ranking','ranking_BERT']])

                                            job_title   ranking  ranking_BERT
12  aspiring human resource management student see...  1.000000      0.399975
16                human resource specialist luxottica  0.716094      1.000000
49                  human resource generalist loparex  0.703923      0.839160
8     seeking human resource hris generalist position  0.658423      0.681701
13                 seeking human resource opportunity  0.580764      0.743979
25                 human conflict policy compensation  0.571468      0.398919
26                   human resource generalist schwan  0.567157      0.877701
17  director human resource north america groupe b...  0.561285      0.652341
15    human resource staffing recruiting professional  0.541803      0.743885
11  human resource coordinator intercontinental bu...  0.535450      0.852687
23  nortia staffing seeking human resource payroll...  0.509338      0.691810
42  seeking human resource opportunity open travel...  0.485328 

In [221]:
# Sort the DataFrame in ascending order based on a column
df_sorted = df.sort_values('ranking_BERT', ascending=False)

# Print the sorted DataFrame
print(df_sorted[['job_title', 'ranking','ranking_BERT', 'connection']])

                                            job_title   ranking  ranking_BERT  \
16                human resource specialist luxottica  0.716094      1.000000   
26                   human resource generalist schwan  0.567157      0.877701   
52         director administration excellence logging  0.203254      0.865002   
51                                 always set success  0.046150      0.855278   
11  human resource coordinator intercontinental bu...  0.535450      0.852687   
49                  human resource generalist loparex  0.703923      0.839160   
19          human resource generalist scottmadden inc  0.482098      0.822557   
3                 people development coordinator ryan  0.130708      0.774029   
1   native english teacher epik english program korea  0.012769      0.745856   
13                 seeking human resource opportunity  0.580764      0.743979   
15    human resource staffing recruiting professional  0.541803      0.743885   
33  rrp brand portfolio exec

# SBERT

In [222]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')


In [223]:
sentences = list(df['job_title'].values)
sentences

['2019 bauer college business graduate magna cum laude aspiring human resource professional',
 'native english teacher epik english program korea',
 'aspiring human resource professional',
 'people development coordinator ryan',
 'advisory board member celal bayar university',
 'aspiring human resource specialist',
 'student humber college aspiring human resource generalist',
 'hr senior specialist',
 'seeking human resource hris generalist position',
 'student chapman university',
 'svp chro marketing communication csr officer engie houston woodland energy gphr sphr',
 'human resource coordinator intercontinental buckhead atlanta',
 'aspiring human resource management student seeking internship',
 'seeking human resource opportunity',
 'experienced retail manager aspiring human resource professional',
 'human resource staffing recruiting professional',
 'human resource specialist luxottica',
 'director human resource north america groupe beneteau',
 'retired army national guard recrui

In [224]:
# Encode the target sentence and other sentences
target_sentence = 'aspiring human resource'
target_embedding = model.encode([target_sentence])[0]
sentence_embeddings = model.encode(sentences)

# Calculate cosine similarity between the target sentence and other sentences
similarities = cosine_similarity([target_embedding], sentence_embeddings)[0]

# Sort the sentences based on their similarity scores
results = sorted(zip(sentences, similarities), key=lambda x: x[1], reverse=True)

# Print the most similar sentences
for sentence, similarity in results:
    print(f"Similarity: {similarity:.4f}\tSentence: {sentence}")

Similarity: 0.7982	Sentence: aspiring human resource professional
Similarity: 0.7982	Sentence: aspiring human resource professional
Similarity: 0.7820	Sentence: seeking human resource opportunity
Similarity: 0.7614	Sentence: aspiring human resource specialist
Similarity: 0.7600	Sentence: seeking human resource position
Similarity: 0.5887	Sentence: aspiring human resource professional energetic leader
Similarity: 0.5831	Sentence: seeking human resource hris generalist position
Similarity: 0.5791	Sentence: aspiring human resource manager seeking internship human resource
Similarity: 0.5609	Sentence: human resource professional
Similarity: 0.5128	Sentence: aspiring human resource management student seeking internship
Similarity: 0.4970	Sentence: aspiring human resource professional passionate helping create inclusive engaging work environment
Similarity: 0.4840	Sentence: director human resource ey
Similarity: 0.4597	Sentence: seeking human resource opportunity open travel relocation
Simil

In [225]:
from sklearn.preprocessing import MinMaxScaler

#Select the input features for ranking
features = ['connection']

# Replace '500+' with 500 in the 'connections' column
df['connection'] = df['connection'].replace('500+ ', '500')

df['connection'] = pd.to_numeric(df['connection'])
print(df['connection'])
# Scale the 'connections' column to a range of 0 to 1
scaler = MinMaxScaler()
df['connection'] = scaler.fit_transform(df['connection'].values.reshape(-1, 1))

# Normalize the remaining features to a range of 0 to 1
df[features] = scaler.fit_transform(df[features])

# Calculate the ranking score
df['ranking_SBERT'] = 0.8*similarities*0.2*df['connection']

# Normalize the ranking score to a range of 0 to 1
df['ranking_SBERT'] = scaler.fit_transform(df['ranking_SBERT'].values.reshape(-1, 1))

# Print the ranked candidates
print(df[['job_title', 'ranking','ranking_SBERT']])


0     0.168337
1     1.000000
2     0.086172
3     1.000000
4     1.000000
5     0.000000
6     0.120240
7     1.000000
8     1.000000
9     0.002004
10    1.000000
11    1.000000
12    1.000000
13    0.779559
14    0.112224
15    1.000000
16    1.000000
17    1.000000
18    0.162325
19    1.000000
20    0.008016
21    0.012024
22    0.030060
23    1.000000
24    0.422846
25    0.817635
26    1.000000
27    0.012024
28    0.102204
29    0.909820
30    0.346693
31    0.535070
32    0.098196
33    1.000000
34    0.006012
35    0.078156
36    0.034068
37    0.697395
38    0.308617
39    0.076152
40    0.126253
41    0.016032
42    0.829659
43    0.112224
44    0.036072
45    0.140281
46    0.006012
47    0.094188
48    0.204409
49    1.000000
50    0.096192
51    1.000000
52    1.000000
Name: connection, dtype: float64
                                            job_title   ranking  ranking_SBERT
0   2019 bauer college business graduate magna cum...  0.098715       0.141204
1   native eng

In [226]:
df.tail(40)

,id,job_title,location,connection,fit,similarities,ranking,aspiring,human,resource,avarage,ranking_BERT,ranking_SBERT
13,28,seeking human resource opportunity,"Chicago, Illinois",0.779559,NaN,0.584583,0.580764,0.946720,0.862424,0.947124,0.896309,0.743979,1.000000
14,66,experienced retail manager aspiring human reso...,"Austin, Texas Area",0.112224,NaN,0.720305,0.103017,0.656040,0.737290,0.653072,0.000000,0.000000,0.122969
15,67,human resource staffing recruiting professional,"Jackson, Mississippi Area",1.000000,NaN,0.425144,0.541803,0.847956,0.879553,0.872207,0.698637,0.743885,0.654748
16,68,human resource specialist luxottica,Greater New York City Area,1.000000,NaN,0.561908,0.716094,0.953045,0.873255,0.963917,0.939173,1.000000,0.450554
17,69,director human resource north america groupe b...,"Greater Grand Rapids, Michigan Area",1.000000,NaN,0.440431,0.561285,0.824047,0.863262,0.844315,0.612661,0.652341,0.624740
18,70,retired army national guard recruiter office m...,"Virginia Beach, Virginia",0.162325,NaN,0.453833,0.093883,0.766718,0.849971,0.790793,0.455915,0.078799,0.152845
19,71,human resource generalist scottmadden inc,"Raleigh-Durham, North Carolina Area",1.000000,NaN,0.378295,0.482098,0.865610,0.903819,0.888803,0.772524,0.822557,0.464661
20,72,business management major aspiring human resou...,"Monroe, Louisiana Area",0.008016,NaN,0.787856,0.008048,0.757936,0.780612,0.756592,0.314067,0.002681,0.069493
21,73,aspiring human resource manager seeking intern...,"Houston, Texas Area",0.012024,NaN,0.878196,0.013457,0.805727,0.841271,0.814900,0.524623,0.006717,0.075727
22,74,human resource professional,Greater Boston Area,0.030060,NaN,0.562084,0.021533,0.877678,0.904746,0.900114,0.803212,0.025708,0.090907
